In [3]:
import pandas as pd
from ast import literal_eval
import numpy as np
import torch
import models.recipe_encoder as recipe
# import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from collections import Counter, OrderedDict
%load_ext autoreload
%autoreload 2

csv_file = 'Data/updated_data_with_lists.csv'
df = pd.read_csv(csv_file, converters={"Cleaned_Ingredients": literal_eval, "Instructions": literal_eval})
print(len(df))
df.head()

13496


,Unnamed: 0,Title,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[Pat chicken dry with paper towels, season all...",miso-butter-roast-chicken-acorn-squash-panzanella,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa..."
1,1,Crispy Salt and Pepper Potatoes,[Preheat oven to 400°F and line a rimmed bakin...,crispy-salt-and-pepper-potatoes-dan-kluger,"[2 large egg whites, 1 pound new potatoes (abo..."
2,2,Thanksgiving Mac and Cheese,[Place a rack in middle of oven; preheat to 40...,thanksgiving-mac-and-cheese-erick-williams,"[1 cup evaporated milk, 1 cup whole milk, 1 ts..."
3,3,Italian Sausage and Bread Stuffing,[Preheat oven to 350°F with rack in middle. Ge...,italian-sausage-and-bread-stuffing-240559,"[1 (¾- to 1-pound) round Italian loaf, cut int..."
4,4,Newton's Law,[Stir together brown sugar and hot water in a ...,newtons-law-apple-bourbon-cocktail,"[1 teaspoon dark brown sugar, 1 teaspoon hot w..."


In [4]:
column = df["Cleaned_Ingredients"]

ingr_list = []
element_list = []
ingr_max = 0
print("Column length", len(column))
for w_list in column:
    tmp =[]
    for str_list in w_list:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(',' ( ').replace(')',' ) ').replace('"','').split()
        ingr_list.extend(formatted)
        new_len = len(formatted)
        tmp.extend(formatted)
        if new_len>ingr_max:
            ingr_max=new_len
    element_list.append(tmp)

print(ingr_list[:10])
print(element_list[0])
print("Len Elements:", len(element_list))
ingr_vocab = set(ingr_list)
ingr_vocab_size = len(ingr_vocab)
print("Size:", ingr_vocab_size)
print("Max:", ingr_max)

Column length 13496
['1', '(', '3½–4-lb', '.', ')', 'whole', 'chicken', '2¾', 'tsp', '.']
['1', '(', '3½–4-lb', '.', ')', 'whole', 'chicken', '2¾', 'tsp', '.', 'kosher', 'salt', ',', 'divided', ',', 'plus', 'more', '2', 'small', 'acorn', 'squash', '(', 'about', '3', 'lb', '.', 'total', ')', '2', 'Tbsp', '.', 'finely', 'chopped', 'sage', '1', 'Tbsp', '.', 'finely', 'chopped', 'rosemary', '6', 'Tbsp', '.', 'unsalted', 'butter', ',', 'melted', ',', 'plus', '3', 'Tbsp', '.', 'room', 'temperature', '¼', 'tsp', '.', 'ground', 'allspice', 'Pinch', 'of', 'crushed', 'red', 'pepper', 'flakes', 'Freshly', 'ground', 'black', 'pepper', '⅓', 'loaf', 'good-quality', 'sturdy', 'white', 'bread', ',', 'torn', 'into', '1', 'pieces', '(', 'about', '2½', 'cups', ')', '2', 'medium', 'apples', '(', 'such', 'as', 'Gala', 'or', 'Pink', 'Lady;', 'about', '14', 'oz', '.', 'total', ')', ',', 'cored', ',', 'cut', 'into', '1', 'pieces', '2', 'Tbsp', '.', 'extra-virgin', 'olive', 'oil', '½', 'small', 'red', 'onion',

In [5]:
df["Cleaned_Ingredients"][0]

['1 (3½–4-lb.) whole chicken',
 '2¾ tsp. kosher salt, divided, plus more',
 '2 small acorn squash (about 3 lb. total)',
 '2 Tbsp. finely chopped sage',
 '1 Tbsp. finely chopped rosemary',
 '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
 '¼ tsp. ground allspice',
 'Pinch of crushed red pepper flakes',
 'Freshly ground black pepper',
 '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
 '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
 '2 Tbsp. extra-virgin olive oil',
 '½ small red onion, thinly sliced',
 '3 Tbsp. apple cider vinegar',
 '1 Tbsp. white miso',
 '¼ cup all-purpose flour',
 '2 Tbsp. unsalted butter, room temperature',
 '¼ cup dry white wine',
 '2 cups unsalted chicken broth',
 '2 tsp. white miso',
 'Kosher salt',
 'freshly ground pepper']

In [6]:
column = df["Instructions"]

inst_list = []
inst_max = 0
for w_list in column:
    for str_list in w_list:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
        inst_list.extend(formatted)
        new_len = len(formatted)
        if new_len>inst_max:
            inst_max=new_len
            
print(inst_list[:10])
inst_vocab = set(inst_list)
inst_vocab_size = len(inst_vocab)
print("Size:",inst_vocab_size)
print("Max:", inst_max)

['Pat', 'chicken', 'dry', 'with', 'paper', 'towels', 'season', 'all', 'over', 'with']
Size: 21164
Max: 247


In [7]:
column = df["Title"].to_list()

title_list = []
title_max = 0
for str_list in column:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
        title_list.extend(formatted)
        new_len = len(formatted)
        if new_len>title_max:
            title_max=new_len

print(title_list[:10])
title_vocab = set(title_list)
title_vocab_size = len(title_vocab)
print("Size:", title_vocab_size)
print("Max:", title_max)

['Miso-Butter', 'Roast', 'Chicken', 'With', 'Acorn', 'Squash', 'Panzanella', 'Crispy', 'Salt', 'and']
Size: 6959
Max: 21


In [8]:
total_vocab = set(list(inst_vocab) + list(ingr_vocab) + list(title_vocab)) 
total_vocab_size = len(total_vocab)
print("Size:",total_vocab_size)
total_max = max([inst_max, ingr_max, title_max])
print("Max:",total_max)

Size: 31551
Max: 247


In [9]:
word_to_ix = {word: i for i, word in enumerate(total_vocab)}

word_to_ix_ingr = {word: i for i, word in enumerate(ingr_vocab)}
word_to_ix_inst = {word: i for i, word in enumerate(inst_vocab)}
word_to_ix_title = {word: i for i, word in enumerate(title_vocab)}

In [10]:
# from torchtext.vocab import vocab

# Get longest sentence in Ingredients or Instructions for max_len
# Use torchtext to make vocab: vocab_size = input_size

In [11]:
tokenizer = get_tokenizer("basic_english")
tokens = tokenizer(' '.join(inst_list))
print(len(inst_list))
# print(inst_list[:30])
print(len(tokens))
print(tokens[:30])

tokenizer = get_tokenizer("basic_english")
tokens = tokenizer(' '.join(inst_vocab))
print(len(inst_vocab))
# print(list(inst_vocab)[:30])
print(len(tokens))
print(tokens[:30])


2644584
2649226
['pat', 'chicken', 'dry', 'with', 'paper', 'towels', 'season', 'all', 'over', 'with', '2', 'tsp', '.', 'salt', 'and', 'tie', 'legs', 'together', 'with', 'kitchen', 'twine', '.', 'let', 'sit', 'at', 'room', 'temperature', '1', 'hour', '.']
21164
21945
['alfalfa', 'may', 'apple-filled', 'puffs', '4-inch', 'thermostatically', 'gray-ashed', 'contracts', 'flourlike', 'dime-size', 'cumin', 'churn', 'tops', 'thanks', 'crumb', 'crumb-coating', 'ranch', 'flex', '22–26', 'versatile', 'rye-crisp', 'separately', 'broth—or', 'drag', 'fried', 'twelve', 'personal', 'stage', 'vailable', 'else']


In [12]:
# Basic
# tokenizer = get_tokenizer("basic_english")
# # from torchtext.transforms import CLIPTokenizer
# # MERGES_FILE = "http://download.pytorch.org/models/text/clip_merges.bpe"
# # ENCODER_FILE = "http://download.pytorch.org/models/text/clip_encoder.json"
# # tokenizer = CLIPTokenizer(merges_path=MERGES_FILE, encoder_json_path=ENCODER_FILE)

# title_words = tokenizer(' '.join(title_list))
# inst_words = tokenizer(' '.join(inst_list))
# ingr_words = tokenizer(' '.join(ingr_list))
# all_words = title_words + inst_words + ingr_words
# print("All words", len(all_words))

# counter = Counter(all_words)
# sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
# ordered_dict = OrderedDict(sorted_by_freq_tuples)
# total_vocab = vocab(counter)
# print("Vocab size:", len(total_vocab))

def get_max_len(list_object):
    max_len = 0
    # Tokenize each sentence
    for sentence in list_object:
        tokens = tokenizer(sentence)
        if len(tokens)>max_len:
            max_len = len(tokens)

    return max_len

inst_max = df["Instructions"].apply(lambda x: get_max_len(x)).max()
ingr_max = df["Cleaned_Ingredients"].apply(lambda x: get_max_len(x)).max()

print("Max Instructions tokens", inst_max)
print("Max Ingredients tokens", ingr_max)

Max Instructions tokens 247
Max Ingredients tokens 110


In [13]:
# import os
# import sys
# import requests
# from tqdm import tqdm

# if len(sys.argv) != 2:
#     print('You must enter the model name as a parameter, e.g.: download_model.py 124M')
#     sys.exit(1)

# model = sys.argv[1]

# subdir = os.path.join('models', model)
# if not os.path.exists(subdir):
#     os.makedirs(subdir)
# subdir = subdir.replace('\\','/') # needed for Windows

# for filename in ['checkpoint','encoder.json','hparams.json','model.ckpt.data-00000-of-00001', 'model.ckpt.index', 'model.ckpt.meta', 'vocab.bpe']:

#     r = requests.get("https://openaipublic.blob.core.windows.net/gpt-2/" + subdir + "/" + filename, stream=True)

#     with open(os.path.join(subdir, filename), 'wb') as f:
#         file_size = int(r.headers["content-length"])
#         chunk_size = 1000
#         with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
#             # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
#             for chunk in r.iter_content(chunk_size=chunk_size):
#                 f.write(chunk)
#                 pbar.update(chunk_size)

In [15]:
tokenizer = get_tokenizer("basic_english")
# from torchtext.transforms import CLIPTokenizer, CharBPETokenizer, GPT2BPETokenizer
# MERGES_FILE = "http://download.pytorch.org/models/text/clip_merges.bpe"
# ENCODER_FILE = "http://download.pytorch.org/models/text/clip_encoder.json"
# tokenizer = CLIPTokenizer(merges_path=MERGES_FILE, encoder_json_path=ENCODER_FILE, num_merges=50)
# tokenizer = CharBPETokenizer(ENCODER_FILE,MERGES_FILE)
# tokenizer = GPT2BPETokenizer("models/gpt_2/encoder.json",
#                              "models/gpt_2/vocab.bpe")

title_words = tokenizer(' '.join(title_list))
inst_words = tokenizer(' '.join(inst_list))
ingr_words = tokenizer(' '.join(ingr_list))
all_words = title_words + inst_words + ingr_words
print("All words", len(all_words))

counter = Counter(all_words)
sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
total_vocab = Vocab(counter)
print("Vocab size:", len(total_vocab))

def get_max_len(list_object):
    max_len = 0
    # Tokenize each sentence
    for sentence in list_object:
        tokens = tokenizer(sentence)
        if len(tokens)>max_len:
            max_len = len(tokens)

    return max_len

inst_max = df["Instructions"].apply(lambda x: get_max_len(x)).max()
ingr_max = df["Cleaned_Ingredients"].apply(lambda x: get_max_len(x)).max()

print("Max Instructions tokens", inst_max)
print("Max Ingredients tokens", ingr_max)

All words 3685814
Vocab size: 23945
Max Instructions tokens 247
Max Ingredients tokens 110


In [16]:
print(tokenizer(' '.join(inst_list))[:70])
print(inst_list[:70])

['pat', 'chicken', 'dry', 'with', 'paper', 'towels', 'season', 'all', 'over', 'with', '2', 'tsp', '.', 'salt', 'and', 'tie', 'legs', 'together', 'with', 'kitchen', 'twine', '.', 'let', 'sit', 'at', 'room', 'temperature', '1', 'hour', '.', 'meanwhile', 'halve', 'squash', 'and', 'scoop', 'out', 'seeds', '.', 'run', 'a', 'vegetable', 'peeler', 'along', 'ridges', 'of', 'squash', 'halves', 'to', 'remove', 'skin', '.', 'cut', 'each', 'half', 'into', '½-thick', 'wedges', 'arrange', 'on', 'a', 'rimmed', 'baking', 'sheet', '.', 'combine', 'sage', 'rosemary', 'and', '6', 'tbsp']
['Pat', 'chicken', 'dry', 'with', 'paper', 'towels', 'season', 'all', 'over', 'with', '2', 'tsp', '.', 'salt', 'and', 'tie', 'legs', 'together', 'with', 'kitchen', 'twine', '.', 'Let', 'sit', 'at', 'room', 'temperature', '1', 'hour', '.', 'Meanwhile', 'halve', 'squash', 'and', 'scoop', 'out', 'seeds', '.', 'Run', 'a', 'vegetable', 'peeler', 'along', 'ridges', 'of', 'squash', 'halves', 'to', 'remove', 'skin', '.', 'Cut', 

In [17]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = 100
# process = TensorDictTokenizer(tokenizer, max_length=10)

In [18]:
def pad_to_value(x, val):
    if x.shape[1]>val:
        raise ValueError(f"{x.shape[1]}>{val}")
    diff = val - x.shape[-1]
    # top, bottom
    pad = (0, diff)
    out = torch.nn.functional.pad(x, pad, mode='constant', value=0)
    return out

In [19]:
# Test recipe encoder:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)


## Dummy data between 0 and 10
# Size: batch=2, title_len = 8, ingr_len=15, instr_len=30
src_list = [
            # title
            np.random.rand(2, 8) * 10,
            # ingredients with 5 lines
            np.random.rand(2, 5, 15) * 10,
             # instructions with 7 lines
            np.random.rand(2, 7, 15) * 10
            ]
# Convert to torch and pad to max length
x = [ pad_to_value(torch.tensor(t, dtype=torch.long),total_max) for t in src_list]
print("Input shapes:")
print(x[0].shape)
print(x[1].shape)
print(x[2].shape)




You are using device: cuda
Input shapes:
torch.Size([2, 247])
torch.Size([2, 5, 247])
torch.Size([2, 7, 247])


In [20]:
recipe_encoder = recipe.RecipeEncoder(device="cuda", vocab_size=total_vocab_size,
                                      max_len=total_max)
outputs = recipe_encoder.forward(x)
print(outputs)

(tensor([[[ 0.0808,  0.4999,  1.3792,  ...,  0.2700,  0.1253, -1.1917],
         [ 0.5416,  0.3807,  0.9894,  ...,  1.1628,  0.0718, -0.3322],
         [ 0.4275, -0.3802,  1.4338,  ..., -0.4330, -0.0523,  0.2777],
         ...,
         [ 0.2969,  1.2286, -0.9710,  ...,  1.3662,  1.8854,  0.7023],
         [-0.9929,  1.0657, -1.0301,  ...,  0.4181,  1.0254,  0.3174],
         [-0.3701,  1.7279, -0.2846,  ...,  1.6393,  2.0879, -0.0570]],

        [[ 0.2215,  0.5239,  2.2554,  ..., -0.0745, -0.7323, -0.7702],
         [ 0.2834,  0.7428,  0.5610,  ...,  0.9669, -0.0091, -1.0082],
         [-0.1378,  1.0216,  0.1937,  ...,  0.6632,  0.1825, -0.7075],
         ...,
         [ 0.1702,  1.2358, -0.6299,  ...,  0.8203,  1.6653,  0.8110],
         [-0.2664,  1.2936, -1.0354,  ...,  1.5330,  1.5045,  0.5183],
         [-0.4329,  1.6971, -0.5873,  ...,  1.2488,  2.0607,  0.0758]]],
       device='cuda:0', grad_fn=<CatBackward0>), tensor([[-0.2227,  0.0957, -0.4042,  ..., -0.4722, -0.2344, -0.617